In [1]:
import os
import glob
from pci.fimport import fimport
from pci.atcor import atcor

In [7]:
# Create the 'generated_pixfile' directory if it doesn't exist
output_dir = os.path.join(os.getcwd(), 'generated_pixfile')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create lists to hold input files for processing
images_mtl = []
parent_directory = os.path.join(os.getcwd(), 'images')
image_folders = os.listdir(parent_directory)

# Iterate through each folder and find the MTL file
for folder in image_folders:
    folder_path = os.path.join(parent_directory, folder)  # Get full path of the folder
    
    # Find all *_MTL.txt files in the current folder (recursive search)
    mtl_files = glob.glob(os.path.join(folder_path, '**', '*_MTL.txt'), recursive=True)
    
    if mtl_files:
        images_mtl.extend(mtl_files)  # Add the found MTL files to the list

# Now process each MTL file and create a PIX file for each inside 'generated_pixfile' folder
for mtl_file in images_mtl:
    # Define output PIX file path inside 'generated_pixfile' folder based on the MTL file name
    pix_file = os.path.join(output_dir, os.path.basename(mtl_file).replace('.txt', '.pix'))  # Replace _MTL.txt with .pix extension
    
    # Create the PIX file using fimport
    try:
        fimport(fili=mtl_file, filo=pix_file)
        print(f"Successfully created PIX file: {pix_file}")
    except Exception as e:
        print(f"Error importing {mtl_file}: {e}")

In [ ]:
# Apply atmospheric correction to each PIX file inside 'generated_pixfile'
generated_pix_files = glob.glob(os.path.join(output_dir, '*.pix'))  # Find all .pix files in 'generated_pixfile'

for pix_file in generated_pix_files:
    try:
        # Define the output PIX file path for the atmospheric corrected file
        atcor_pix_file = pix_file.replace('.pix', '_Atcor.pix')
        
        # Define the necessary input files for atmospheric correction
        # (you must define or provide paths for these files)
        masks = "path_to_mask_file.pix"  # Provide the path to the mask file
        ter_file = "path_to_terrain_file.dem"  # Provide the path to the terrain file (e.g., DEM)
        illum_file = "path_to_illumination_file.illum"  # Provide the path to the illumination file

        # Apply atmospheric correction to the PIX file
        atcor(fili=pix_file, 
              maskfili=masks, 
              terfile=ter_file, 
              illufile=illum_file, 
              atmdef="rural", 
              atmcond="Fall", 
              outunits="Percent_Reflectance", 
              filo=atcor_pix_file)
        
        print(f"Successfully applied atmospheric correction and saved to: {atcor_pix_file}")
    
    except Exception as e:
        print(f"Error applying atmospheric correction to {pix_file}: {e}")

In [ ]:
# Apply atmospheric correction to each PIX file inside 'generated_pixfile'
generated_pix_files = glob.glob(os.path.join(output_dir, '*.pix'))  # Find all .pix files in 'generated_pixfile'

for pix_file in generated_pix_files:
    try:
        # Define the output PIX file path for the atmospheric corrected file
        atcor_pix_file = pix_file.replace('.pix', '_Atcor.pix')
        
        # Define the necessary input files for atmospheric correction
        # (you can set paths to your mask file if available, otherwise None)
        masks = None  # If no mask is available, pass None
        ter_file = None  # No terrain file, passing None
        illum_file = None  # No illumination file, passing None

        # Apply atmospheric correction to the PIX file
        atcor(fili=pix_file, 
              maskfili=masks, 
              terfile=ter_file, 
              illufile=illum_file, 
              atmdef="rural", 
              atmcond="Fall", 
              outunits="Percent_Reflectance", 
              filo=atcor_pix_file)
        
        print(f"Successfully applied atmospheric correction and saved to: {atcor_pix_file}")
    
    except Exception as e:
        print(f"Error applying atmospheric correction to {pix_file}: {e}")